In [43]:
# 查询测试

import MySQLdb
import elasticsearch
from elasticsearch.helpers import bulk, streaming_bulk
from IPython.display import display, HTML
from elasticsearch_dsl import F, Q, Search

es = elasticsearch.Elasticsearch('http://172.16.1.30:9200')
INDEX = 'meila_wares'
TYPE = 'wares'


#q = raw_input('请输入关键词：')
q = "玛尔斯"


if not q: q = u'美白 面膜'

s = Search(es, index=INDEX, doc_type=TYPE)

# 查询 DSL
s = s.query(    
Q('bool', should=[Q('constant_score', boost=1.0, query=Q('match', name=dict(query=q, operator='or')))
    , Q('constant_score', boost=0.2, query=Q('match', short_name=dict(query=q, operator='or')))
    , Q('constant_score', boost=0.1, query=Q('match', short_intro=dict(query=q, operator='or')))], disable_coord=True)
)

s = s.params(explain=True)

# 排序
#s = s.sort({"_script": {
#            "script": "_score * 68.0 + doc['display_score'].value * 0.5 + doc['sale_score'].value * 0.3 + doc['comment_score'].value * 0.2",
#            "type": "number", "order": "desc", "lang": "groovy"}})

# 分页
s = s[:10]


display(s.to_dict())

r = s.execute()

htmls = [u'<br><br>'
         u'<table>'
        u'<tr>'
        u'<th>#</th>'
        u'<th>ID</th>'
        u'<th>标题</th>'
        u'<th>短标题</th>'
        u'<th>短描述</th>'
        u'<th>销量分数</th>'
        u'<th>点评分数</th>'
        u'<th>设置的分数</th>'
        u'<th>ES分数</th>'
         u'<th>解释</th>'
       ]

for idx, doc in enumerate(r):
     htmls.extend([u'<tr>'
                  u'<td>{idx}</td>'.format(idx=idx+1),
                  u'<td>{id}</td>'.format(id=doc.id),
                  u'<td>{name}</td>'.format(name=doc.name),
                  u'<td>{short_name}</td>'.format(short_name=doc.short_name),
                  u'<td>{short_intro}</td>'.format(short_intro=doc.short_intro),
                  u'<td>{sale_score}</td>'.format(sale_score=doc.sale_score),
                  u'<td>{comment_score}</td>'.format(comment_score=doc.comment_score),
                  u'<td>{display_score}</td>'.format(display_score=doc.display_score),
                  u'<td>{last_score}</td>'.format(last_score=doc.meta.score),
                  u'<td>{explanation}</td>'.format(explanation=', '.join([str(x.value) for x in doc.meta.explanation.details]))]
                 )

display(HTML(''.join(htmls)))

{'from': 0,
 'query': {'bool': {'disable_coord': True,
   'should': [{'constant_score': {'boost': 1.0,
      'query': {'match': {'name': {'operator': 'or',
         'query': '\xe7\x8e\x9b\xe5\xb0\x94\xe6\x96\xaf'}}}}},
    {'constant_score': {'boost': 0.2,
      'query': {'match': {'short_name': {'operator': 'or',
         'query': '\xe7\x8e\x9b\xe5\xb0\x94\xe6\x96\xaf'}}}}},
    {'constant_score': {'boost': 0.1,
      'query': {'match': {'short_intro': {'operator': 'or',
         'query': '\xe7\x8e\x9b\xe5\xb0\x94\xe6\x96\xaf'}}}}}]}},
 'size': 10}

#,ID,标题,短标题,短描述,销量分数,点评分数,设置的分数,ES分数,解释
1,3484,神烦大叔卡积分卡洛斯,神烦大叔卡积分卡洛斯,神烦大叔卡积分卡洛斯,0,0,0,1.2686702,"0.9759001, 0.19518003, 0.097590014"
2,3425,韩国菲斯小铺绿豆泥浆面膜,韩国菲斯小铺绿豆泥,韩国菲斯小铺绿豆泥浆面膜,0,0,0,1.2686702,"0.9759001, 0.19518003, 0.097590014"
3,1034,JUNPING橙净无忧氨基酸控油洁面慕斯135ml <90元包邮>,俊平橙净洁面慕斯,让你快乐的洁面慕斯,2,62,0,1.2686702,"0.9759001, 0.19518003, 0.097590014"
4,2028,山东大樱桃”拉宾斯“优质车厘子3斤<128元包空运>,山东大樱桃拉宾斯,补铁补血补钙补维生素,35,60,0,1.1710801,"0.9759001, 0.19518003"
5,2726,法国Bioderma贝德玛舒妍温和卸妆水 500ml <115元包邮>,贝德玛卸妆水,卸除彩妆 温和清洁,58,60,0,1.1710801,"0.9759001, 0.19518003"
6,2856,韩国RECIPE菜斯璧水晶透明防晒喷雾150ml 68包邮,菜斯璧水晶防晒喷雾,防晒防水防汗定妆保湿,62,60,0,1.1710801,"0.9759001, 0.19518003"
7,548,韩国Etude house爱丽小屋慕斯染色唇彩三选一10g <58包邮>,爱丽小屋慕斯染色唇彩,挡不住的唇色诱惑,0,80,0,1.1710801,"0.9759001, 0.19518003"
8,742,【海淘专场】日本 Albion澳尔滨药用化妆水 健康水 110ml <260包邮>,日本澳尔滨健康水,无敌保湿王,0,66,0,1.1710801,"0.9759001, 0.19518003"
9,1274,Miss Candy健康指彩 卡梅尔栗色彩点 16ml<55包邮>,卡梅尔栗色彩点,五毒无味健康可剥,10,60,0,1.1710801,"0.9759001, 0.19518003"
10,1311,【美啦闪购仅限一天】法国BIODERMA贝德玛卸妆水普通粉水|特润粉水|蓝水三选一<最低108包邮>,贝德玛卸妆水,开创卸妆新“净”界,11,88,0,1.1710801,"0.9759001, 0.19518003"


In [67]:
# 查询测试

import MySQLdb
import elasticsearch
from elasticsearch.helpers import bulk, streaming_bulk
from IPython.display import display, HTML
from elasticsearch_dsl import F, Q, Search

es = elasticsearch.Elasticsearch('http://172.16.1.30:9200')
INDEX = 'meila_wares'
TYPE = 'wares'


#q = raw_input('请输入关键词：')
q = "美白柔肤水"


if not q: q = u'美白 面膜'

s = Search(es, index=INDEX, doc_type=TYPE)

# 查询 DSL
#s = s.query(    
#Q('bool', should=[Q('constant_score', boost=1.0, query=Q('match', name=dict(query=q, operator='or')))]
#                  , minimum_should_match=3)
#)

#s = s.query("match", name=dict(query=q, operator='or')).query('match', short_name=dict(query=q, operator='or'))
s = s.query(Q('bool', must=[Q('match', name=dict(query=q, operator='or'))], boost=1, disable_coord=True) |
            Q('bool', must=[Q('match', short_name=dict(query=q, operator='or'))], boost=0.2, disable_coord=True) |
           Q('bool', must=[Q('match', short_intro=dict(query=q, operator='or'))], boost=0.1, disable_coord=True))

#s = s.query(Q('multi_match', fields=["name^10", "short_name^2", "short_intro"], query=q, tie_breaker=0.99, operator="or"))

#s = s.query(Q('dis_max', tie_breaker=1.0, queries=[Q('match', name=dict(query=q, operator='or')),
#                                                   Q('match', short_name=dict(query=q, operator='or')),
#                                                  Q('match', short_intro=dict(query=q, operator='or')),]))

#s = s.query(Q('dismax', fields=["name^10", "short_name^2", "short_intro"], query=q, tie_breaker=1))

s = s.params(explain=True)

 #排序
#s = s.sort({"_script": {
#            "script": "_score * 68.0 + doc['display_score'].value * 0.5 + doc['sale_score'].value * 0.3 + doc['comment_score'].value * 0.2",
#            "type": "number", "order": "desc", "lang": "groovy"}})

# 分页
s = s[:20]

#import json
#import pprint
#pprint.pprint(json.dumps(s.to_dict()))
display(s.to_dict())

r = s.execute()

htmls = [u'<br><br>'
         u'<table>'
        u'<tr>'
        u'<th>#</th>'
        u'<th>ID</th>'
        u'<th>标题</th>'
        u'<th>短标题</th>'
        u'<th>短描述</th>'
        u'<th>销量分数</th>'
        u'<th>点评分数</th>'
        u'<th>设置的分数</th>'
        u'<th>ES分数</th>'
         u'<th>解释</th>'
       ]

for idx, doc in enumerate(r):
     htmls.extend([u'<tr>'
                  u'<td>{idx}</td>'.format(idx=idx+1),
                  u'<td>{id}</td>'.format(id=doc.id),
                  u'<td>{name}</td>'.format(name=doc.name),
                  u'<td>{short_name}</td>'.format(short_name=doc.short_name),
                  u'<td>{short_intro}</td>'.format(short_intro=doc.short_intro),
                  u'<td>{sale_score}</td>'.format(sale_score=doc.sale_score),
                  u'<td>{comment_score}</td>'.format(comment_score=doc.comment_score),
                  u'<td>{display_score}</td>'.format(display_score=doc.display_score),
                  u'<td>{es_score}</td>'.format(es_score=doc.meta.score),
                  u'<td>{explanation}</td>'.format(explanation=', '.join([str(x.value) for x in doc.meta.explanation.details]))]
                 )

display(HTML(''.join(htmls)))

{'from': 0,
 'query': {'bool': {'should': [{'bool': {'boost': 1,
      'disable_coord': True,
      'must': [{'match': {'name': {'operator': 'or',
          'query': '\xe7\x8e\x9b\xe5\xb0\x94\xe6\x96\xaf'}}}]}},
    {'bool': {'boost': 0.2,
      'disable_coord': True,
      'must': [{'match': {'short_name': {'operator': 'or',
          'query': '\xe7\x8e\x9b\xe5\xb0\x94\xe6\x96\xaf'}}}]}},
    {'bool': {'boost': 0.1,
      'disable_coord': True,
      'must': [{'match': {'short_intro': {'operator': 'or',
          'query': '\xe7\x8e\x9b\xe5\xb0\x94\xe6\x96\xaf'}}}]}}]}},
 'size': 20}

#,ID,标题,短标题,短描述,销量分数,点评分数,设置的分数,ES分数,解释
1,1924,Marvis玛尔斯 美白薄荷牙膏,Marvis玛尔斯牙,牙膏中的爱马仕,31,60,0,2.0173545,"3.0260317, 0.6666667"
2,2776,日本吉尔•斯图尔特四色腮红8g1#/5# 多款可选 238包邮,吉尔•斯图尔特腮红,打造粉嫩好气色,59,60,0,0.9604768,"1.4407152, 0.6666667"
3,383,【奢华美】日本吉尔•斯图尔特四色腮红8g1#/5# 多款可选 238包邮,吉尔•斯图尔特腮红,打造粉嫩好气色,0,90,0,0.8505046,"1.2757568, 0.6666667"
4,3484,神烦大叔卡积分卡洛斯,神烦大叔卡积分卡洛斯,神烦大叔卡积分卡洛斯,0,0,0,0.49211618,"0.3785509, 0.07571019, 0.037855096"
5,301,日本 JILL STUART吉尔•斯图尔特5#甜心恋爱四色腮红8g<238包邮>,甜心恋爱四色腮红,公主腮红盘驾到,0,68,0,0.37534937,"1.126048, 0.33333334"
6,231,日本 JILL STUART吉尔•斯图尔特5#甜心恋爱四色腮红8g<238包邮>,,,0,62,0,0.3250718,"0.97521544, 0.33333334"
7,2726,法国Bioderma贝德玛舒妍温和卸妆水 500ml <115元包邮>,贝德玛卸妆水,卸除彩妆 温和清洁,58,60,0,0.309005,"0.46350744, 0.6666667"
8,3425,韩国菲斯小铺绿豆泥浆面膜,韩国菲斯小铺绿豆泥,韩国菲斯小铺绿豆泥浆面膜,0,0,0,0.27748716,"0.19994381, 0.05569262, 0.021850713"
9,292,【情人节专场】日本 JILL STUART吉尔•斯图尔特5#甜心恋爱四色腮红8g<238包邮>,,,0,60,0,0.27291217,"0.8187365, 0.33333334"
10,271,【年度打折季】?牛尔老师推荐?贝德玛舒妍洁肤液500ml粉水,,,0,94,0,0.26929832,"0.80789495, 0.33333334"
